In [16]:
import json
import os
import re
import sys
import typing as T

import dotenv
import googlemaps
import nltk
import pandas as pd
from geopy.geocoders import Nominatim
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from openai import OpenAI

dotenv.load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")

CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)
SRC_DIR = os.path.join(ROOT_DIR, "src")

sys.path.append(SRC_DIR)

In [17]:
nltk.download("averaged_perceptron_tagger")
nltk.download("punkt")
nltk.download("stopwords")

gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ross/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/ross/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ross/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
METERS_PER_MILE = 1609.34


def miles_to_meters(miles: float) -> float:
    return miles * METERS_PER_MILE


def get_city_center_coordinates(city_name: str) -> T.Optional[T.Tuple[float, float]]:
    geolocator = Nominatim(user_agent="tgtg")

    location = geolocator.geocode(city_name)

    if not location:
        return None

    return (location.latitude, location.longitude)


def clean_text(text: str) -> str:
    """
    Function to remove stop words, punctuation, and double quotes
    """
    text = text.lower()
    text = text.replace('"', "")
    # Tokenize the text into words
    words = word_tokenize(text)
    # Remove punctuation
    words = [word for word in words if word.isalnum()]
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word not in stop_words]
    cleaned_text = " ".join(filtered_words)
    return cleaned_text


def parse_itenerary_day(lines: T.List[str]) -> T.List[T.Tuple[str, str]]:
    day_plan = []
    # This pattern is designed to capture two groups separated by various delimiters
    pattern = re.compile(r"\-?\s*([\w\s]+?)\s*(?::|at|-)\s*([\w\s'&]+)")

    for line in lines:
        match = pattern.match(line)
        if match:
            activity_type, place = match.groups()
            activity_type = activity_type.strip()
            place = place.split("(")[0].strip()  # Removes anything within parentheses
            day_plan.append((activity_type, place))
        else:
            print(f"Could not parse line: {line}")

    return day_plan


def parse_itenerary_content(content: str) -> T.List[T.Dict[str, str]]:
    # Split the content into days and activities
    days = content.split("\n\n")
    data = []
    for day in days:
        lines = day.split("\n")
        day_number = lines[0].split(" ")[1]
        day_plan = parse_itenerary_day(lines[1:])
        for activity_type, place in day_plan:
            data.append({"Day": day_number, "Activity Type": activity_type, "Place": place})

    return data

In [19]:
inputs = {
    "location": "South Beach Miami, FL",
    "number_of_people": 6,
    "date": "November 2026",
    "duration_days": 3,
    "group_type": "bachelorette party",
    "description": (
        "include boutique hotel options must 4 stars higher onsite spa beach clubs djs"
        "day high end nightclubs list least six restaurant options dinner include least "
        "one nice steakhouse one nice sushi restaurant"
    ),
}

In [20]:
PROMPT_START = """Create itinerary: provide only names
for places listed in google places for breakfast,
morning activity, lunch, afternoon activity, dinner and
evening activity. return only the day and the name:
""".replace(
    "\n", " "
)

PROMPT_TEMPLATE = f"""“{inputs['duration_days']}” day
“{inputs['group_type']}” to “{inputs['location']}” for “{inputs['number_of_people']}”
people in “{inputs['date']}” that enjoy “{inputs['description']}”
"""

prompt = f"{PROMPT_START}{clean_text(PROMPT_TEMPLATE)}"
print(prompt)

Create itinerary: provide only names for places listed in google places for breakfast, morning activity, lunch, afternoon activity, dinner and evening activity. return only the day and the name: 3 day bachelorette party south beach miami fl 6 people november 2026 enjoy include boutique hotel options must 4 stars higher onsite spa beach clubs djsday high end nightclubs list least six restaurant options dinner include least one nice steakhouse one nice sushi restaurant


In [21]:
client = OpenAI(api_key=OPEN_AI_API_KEY)


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)
print(response.choices[0].message.content.split("\n"))

# Extract token usage from the response
completion_tokens = response.usage.completion_tokens
prompt_tokens = response.usage.prompt_tokens
total_tokens = completion_tokens + prompt_tokens

# Cost estimation for 1,000,000 requests
total_requests = 1_000_000
total_tokens_for_all_requests = total_tokens * total_requests

training_cost_per_million_tokens = 8.00
input_usage_cost_per_million_tokens = 3.00
output_usage_cost_per_million_tokens = 6.00

total_cost = (total_tokens_for_all_requests / 1_000_000) * (
    training_cost_per_million_tokens
    + input_usage_cost_per_million_tokens
    + output_usage_cost_per_million_tokens
)

print(f"Estimated total cost for {total_requests} requests: ${total_cost:.2f}")

['Day 1:', '- Breakfast: Starlite Cafe', '- Morning activity: South Pointe Park', '- Lunch: Yardbird Southern Table & Bar', '- Afternoon activity: Miami Beach Boardwalk', '- Dinner: Prime 112', '- Evening activity: Story Nightclub', '', 'Day 2:', '- Breakfast: Big Pink', '- Morning activity: Lincoln Road Mall', "- Lunch: Joe's Stone Crab", '- Afternoon activity: Lummus Park Beach', '- Dinner: Nobu Miami', '- Evening activity: LIV Nightclub', '', 'Day 3:', '- Breakfast: Icebox Cafe', '- Morning activity: Art Deco Historic District', '- Lunch: Prime Italian', '- Afternoon activity: Miami Seaquarium', '- Dinner: STK Miami Beach', '- Evening activity: WALL Lounge']
Estimated total cost for 1000000 requests: $4403.00


In [22]:
# Extract the itinerary content from the response
itinerary_content = response.choices[0].message.content
data = parse_itenerary_content(itinerary_content)
df = pd.DataFrame(data)

print(df)

   Day       Activity Type                          Place
0   1:           Breakfast                  Starlite Cafe
1   1:    Morning activity              South Pointe Park
2   1:               Lunch  Yardbird Southern Table & Bar
3   1:  Afternoon activity          Miami Beach Boardwalk
4   1:              Dinner                      Prime 112
5   1:    Evening activity                Story Nightclub
6   2:           Breakfast                       Big Pink
7   2:    Morning activity              Lincoln Road Mall
8   2:               Lunch               Joe's Stone Crab
9   2:  Afternoon activity              Lummus Park Beach
10  2:              Dinner                     Nobu Miami
11  2:    Evening activity                  LIV Nightclub
12  3:           Breakfast                    Icebox Cafe
13  3:    Morning activity     Art Deco Historic District
14  3:               Lunch                  Prime Italian
15  3:  Afternoon activity               Miami Seaquarium
16  3:        

In [23]:
city_coordinates = get_city_center_coordinates(inputs["location"])
print(f"{inputs['location']} coordinates: {city_coordinates}")
keyword = "breakfast"

itinerary_place_details = []
nearby_place_details = {}
for index, row in df.iterrows():
    try:
        result = gmaps.places(query=row["Place"], location=city_coordinates)
    except:
        print(f"Unable to get places info for {row['Place']}")
        continue

    if not result or result.get("status") != "OK":
        print(f"Unable to get places info for {row['Place']}")
        continue

    place_result = result["results"][0]
    itinerary_place_details.append(place_result)

    radius_meters = miles_to_meters(1.0)

    print(f"Getting nearby places for {row['Place']} at {place_result['geometry']['location']}")
    try:
        nearby_places = gmaps.places_nearby(
            location=place_result["geometry"]["location"],
            radius=radius_meters,
            keyword=keyword if keyword else None,
            type=place_result["types"][0] if place_result["types"] else None,
        )
    except:
        print(f"Unable to get nearby places info for {row['Place']}")
        continue

    if not nearby_places or nearby_places.get("status") != "OK":
        print(f"Unable to get nearby places info for {row['Place']}")
        continue

    nearby_place_details[row["Place"]] = nearby_places["results"]

South Beach Miami, FL coordinates: (25.7744291, -80.1332415)
Getting nearby places for Starlite Cafe at {'lat': 25.7777202, 'lng': -80.13147459999999}
Getting nearby places for South Pointe Park at {'lat': 25.7663809, 'lng': -80.134671}
Unable to get nearby places info for South Pointe Park
Getting nearby places for Yardbird Southern Table & Bar at {'lat': 25.7890685, 'lng': -80.14015789999999}
Getting nearby places for Miami Beach Boardwalk at {'lat': 25.8201212, 'lng': -80.1208166}
Unable to get nearby places info for Miami Beach Boardwalk
Getting nearby places for Prime 112 at {'lat': 25.76992, 'lng': -80.13326699999999}
Getting nearby places for Story Nightclub at {'lat': 25.7705572, 'lng': -80.1340678}
Getting nearby places for Big Pink at {'lat': 25.7708433, 'lng': -80.1336919}
Getting nearby places for Lincoln Road Mall at {'lat': 25.7906517, 'lng': -80.13657599999999}
Unable to get nearby places info for Lincoln Road Mall
Getting nearby places for Joe's Stone Crab at {'lat': 25

In [24]:
print(f"Num results: {len(itinerary_place_details)}")
for item in itinerary_place_details:
    print(item["name"])
    nearby_names = [i["name"] for i in nearby_place_details.get(item["name"], [])]
    for name in nearby_names:
        print(f"  - {name}")

Num results: 18
Starlite Cafe
  - Cafe Bonjour
  - Wilde on the Porch
  - Salt Kitchen & Lounge
  - Eggstaurant
  - Toast All Day
  - Cafe Americano
  - Bacon Bitch
  - FL Cafe
  - Front Porch Cafe
  - Taste Cafe
  - 11th Street Diner
  - Sunny Side Cafe & Eatery
  - Dulce Cafeteria
  - Cafe Ole
  - McDonald's
  - Las Olas Cafe
  - News Cafe
  - Starlite Cafe
  - Plethore et Balthazar
  - The Bistro - Eat. Drink. Connect.®
South Pointe Park
Yardbird Table & Bar
Miami Beach Boardwalk
Prime 112 Restaurant
STORY
Big Pink
  - Cafe Bonjour
  - Salt Kitchen & Lounge
  - Toast All Day
  - Cafe Americano
  - Bacon Bitch
  - FL Cafe
  - 11th Street Diner
  - Sunny Side Cafe & Eatery
  - Dulce Cafeteria
  - Las Olas Cafe
  - News Cafe
  - Starlite Cafe
  - Plethore et Balthazar
  - TGI Fridays
  - Grand Cafe Miami
  - Big Pink
  - South Pointe Tavern
  - Front Porch Cafe
  - Pura Vida
  - Café Bastille South Beach
Lincoln Rd Mall
Joe's Stone Crab
  - The Shoppes at West Avenue
Lummus Park
Nobu M